In [1]:
import os, shutil
import io
from io import StringIO
import re
import sys
from contextlib import redirect_stdout
from keras import models
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd

import numpy as np
import scipy.optimize as opt
import scipy.stats as sp
from scipy.stats import kde
import importlib as im
from sklearn import metrics
import csv

import cnn_utilities as cn
import uq_utilities_2 as uq

Using TensorFlow backend.


In [109]:
####### MTBD ###############

#### multitype bds model
# load trained model and normalization values
mtbd_mymodel = models.load_model("../saved_models/MTBD_128batch_20epoch.hdf5")
mtbd_mean_sd = pd.read_csv("../saved_models/train_mtbd_128batch_20epoch_normalization_label_mean_sd.csv", 
                     index_col = 0).to_numpy()

In [117]:
mtbd_train_means = mtbd_mean_sd[0,:]
mtbd_train_sd = mtbd_mean_sd[1,:]
mtbd_train_aux_priors_means = mtbd_train_means[7:,]
mtbd_train_aux_priors_sd = mtbd_train_sd[7:,]

num_locs = 5
max_tips = 502

In [118]:
train_mtbd_data = pd.read_csv("../data_files/training_mtbd.cblv.csv", 
                   header =None, error_bad_lines = False, index_col = 0).to_numpy()


In [119]:
train_mtbd_labels = pd.read_csv("../data_files/training_mtbd_labels.csv",
                    header = None, error_bad_lines = False).to_numpy()

In [121]:
# num tips and mu data
mtbd_data_mu_ntips = pd.read_table("data_files/labels_and_preds/mtbd_param_values_numtips_propsamp.txt", 
                                  header = 0).to_numpy()
mtbd_data_mu_ntips = np.column_stack((mtbd_data_mu_ntips[:,0], mtbd_data_mu_ntips[:,1]/mtbd_data_mu_ntips[:,2]))

In [122]:
# log R0 sample rate and migration rate
train_mtbd_labels[:,5:12] = np.log(train_mtbd_labels[:,5:12])

# setting up the data shape params and aux params
num_locs = 5
max_tips = 502
num_sample = train_mtbd_data.shape[0]

num_tips = cn.get_num_tips(train_mtbd_data)
subsample_prop = train_mtbd_data[:,(max_tips-1) * 7]
mu = train_mtbd_data[:,(max_tips - 3) * 7]

In [123]:
print(train_mtbd_data.shape)
print(train_mtbd_labels.shape)

(193110, 3514)
(193110, 14)


In [124]:
# normalize labels
labels = train_mtbd_labels[:,5:12]


# set up auxilliary priors and normalize; tree statistics and prior known parameter values (or ranges) 
# get num_tips and normalize
aux_priors_treestats = np.vstack((mu, subsample_prop, num_tips, 
                                  train_mtbd_labels[0:num_sample,12], 
                                  train_mtbd_labels[0:num_sample,13])).transpose()
norm_train_aux_priors_treestats, train_aux_priors_means, train_aux_priors_sd = cn.normalize(aux_priors_treestats)



# create data tensors
        
full_treeLocation_tensor, full_prior_tensor = cn.create_data_tensors(data = train_mtbd_data[0:num_sample,:], 
                                                                        mu = norm_train_aux_priors_treestats[:,0],
                                                                        subsample_prop = norm_train_aux_priors_treestats[:,1],
                                                                        num_tips = norm_train_aux_priors_treestats[:,2],
                                                                        tmrca = norm_train_aux_priors_treestats[:,3],
                                                                        mean_bl = norm_train_aux_priors_treestats[:,4],
                                                                        num_locs = num_locs,
                                                                        max_tips = max_tips,
                                                                        cblv_contains_mu_rho = True)


print(full_treeLocation_tensor.shape, full_prior_tensor.shape)


(193110, 499, 7) (193110, 5, 2)


In [125]:
# make preds
# PREDICT R0_i, sample rate, migration rate
test_mtbd_normalized_preds = mtbd_mymodel.predict([full_treeLocation_tensor, full_prior_tensor])

In [126]:
# reversing normalization
test_cnn_mtbd_preds = cn.denormalize(test_mtbd_normalized_preds, mtbd_train_means[0:7], mtbd_train_sd[0:7])

test_cnn_mtbd_preds = np.column_stack((test_cnn_mtbd_preds[:,0:5], mtbd_data_mu_ntips))


In [128]:
mtbd_tree_preds = pd.read_table("../output/mtbd_nadeau2021_cnn_preds_full_and_a2.tsv",
                             header = 0, error_bad_lines = False).to_numpy()
mtbd_tree_preds = np.column_stack((np.log(mtbd_tree_preds[:,0:5]), [0.05,0.05], [145,93]))
fulltree_R0_preds = mtbd_tree_preds[0][0:5]
a2_R0_preds = mtbd_tree_preds[1][0:5]

In [133]:
import time


nn = test_cnn_mtbd_preds.shape[0]

fulltree_q = []
a2_q = []
for k in range(5):
    start_time = time.time()
    m, lower_q, upper_q = uq.get_CPI(test_cnn_mtbd_preds[0:nn,[k,5,6]], labels[0:nn,k], frac=0.05, inner_quantile=0.95)
    q = (lower_q(mtbd_tree_preds[:,[k,5,6]]), upper_q(mtbd_tree_preds[:,[k,5,6]]))    
    
    fulltree_q.append((q[0][0], q[1][0]))  
    a2_q.append((q[0][1], q[1][1]))
    
    print(str(k) + " complete. time: " + str(time.time() - start_time))

print(fulltree_q)
print(a2_q)

ERROR! Session/line number was not unique in database. History logging moved to new session 3583
0 complete. time: 778.6270086765289
1 complete. time: 791.5997529029846
2 complete. time: 794.1539669036865
3 complete. time: 758.9245924949646
4 complete. time: 764.7643220424652
[(0.7343874837402257, 2.171132993213971), (1.8610547435058888, 5.141123125398311), (1.5999830221621079, 4.28523398388738), (1.2135657797670807, 3.427455760866178), (1.729673311458659, 4.690564784262948)]
[(0.8787991436790704, 2.5758834786684566), (2.1577433254191805, 5.783425265993652), (1.0269365969073618, 2.8818193636558753), (1.4265221390890723, 3.970213434935342), (1.934718098419388, 5.183774367836716)]


In [134]:
# write to file
fulltree_q_a2_q = np.hstack((np.array(fulltree_q).reshape((5,2)), np.array(a2_q).reshape((5,2))))
df_mtbd_ci = pd.DataFrame(fulltree_q_a2_q, index = ["R_" + str(i) for i in range(5)], 
                               columns = ["fulltree_lower_q", "fulltree_upper_q", "A2_lower_q", "A2_upper_q"])
df_mtbd_ci.to_csv('../output/nadeau2021_mtbd_95ci.tsv', sep = "\t", index = True)